In [ ]:

import pymc3 as pm
import pandas as pd
import numpy as np
import theano.tensor as tt
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import math
from sklearn.metrics import mean_squared_error
import plotly.graph_objects as go

In [ ]:
print(pm.__version__)

In [ ]:
mgd= pd.read_csv('../input/merged-data/merged_data.csv')
t=["3728e61d37fa8a73","70ad3d3b82cd4d5d","45d0ab371caa8ad4","577e272c463d42f5","a4df997617cfb6e9"]
mgd=mgd[mgd["unique_id"]==t[0]]

In [ ]:
mgd['change'] = mgd['y'].pct_change()

In [ ]:
mgd['logchange'] = np.log(mgd['y'] / mgd['y'].shift(1))
print(len(mgd))

In [ ]:
train_set = mgd[9500:10000]
test_set = mgd[10000:10000+48]
train_set2=mgd[9000:10000]
train_set3=train_set.iloc[::2]
len(test_set)

In [ ]:
with pm.Model() as model:
#prior
    sigma = pm.Exponential('sigma', 50, testval=.1)
    mu = pm.Normal('mu', 0, sigma=3, testval=.1)
    nu = pm.Exponential('nu', 0.15)
    logs = pm.GaussianRandomWalk('logs',tau=sigma**-2, shape=500)
#likelihood
    r = pm.StudentT('r',nu, mu=mu, lam=1/np.exp(-2*logs), observed=train_set['logchange'].values)
with model:
    start = pm.find_MAP(vars=[logs, mu, nu,sigma], method="L-BFGS-B")

with model:
    step = pm.NUTS(vars=[logs, mu, nu,sigma] ,scaling=start, gamma=.25)
    start2 = pm.sample(100, step, start=start, progressbar=True)[-1]
    step = pm.NUTS(vars=[logs, mu, nu,sigma],scaling=start2, gamma=.55)
    trace = pm.sample(100,step, start=start2, progressbar=True)
with model:
    predictive_samples = pm.sample_posterior_predictive(trace, samples=48)



In [ ]:
preds_gr = list(predictive_samples.values())

In [ ]:
import matplotlib.pyplot as plt
preds_gr = list(predictive_samples.values())[0]
lower = np.percentile(preds_gr, 2.5, axis=1)
upper = np.percentile(preds_gr, 97.5, axis=1)
preds = np.mean(preds_gr, axis=1)
u=mgd["y"].values[10000]
predclose=[]
for i in range(len(preds)):
    u=u*(math.exp(preds[i]))
    predclose.append(u)
mse = mean_squared_error(test_set['y'].values, predclose)
print(mse)
# Set figure size and style
plt.figure(figsize=(12, 6))
plt.style.use('seaborn-whitegrid')

# Plot historical data
plt.plot(range(48), test_set['logchange'].values, marker='o', label='historical', color='#39304A')

# Plot predictions
plt.plot(range(48), preds, marker='o', label='prediction mean', color='#FFAA00')
plt.plot(range(48), lower, marker='.', label='prediction lower bound', color='#FF0000')
plt.plot(range(48), upper, marker='.', label='prediction upper bound', color='#FF0000')
# Labels and title
plt.title('MCMC prediction on log of change', fontsize=16)
plt.xlabel('Timestep', fontsize=12)
plt.ylabel('log change', fontsize=12)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=2)
plt.grid(True, color='lightgrey')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.style.use('seaborn-whitegrid')

# Plot historical data
plt.plot(range(48), test_set['y'].values, marker='o', label='historical', color='#39304A')

# Plot predictions
plt.plot(range(48), predclose, marker='o', label='predictions', color='#FFAA00')

# Labels and title
plt.title('MCMC prediction on electronic demand', fontsize=16)
plt.xlabel('Timestep', fontsize=12)
plt.ylabel('electronic demand', fontsize=12)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=2)
plt.grid(True, color='lightgrey')

plt.tight_layout()
plt.show()

In [ ]:
with pm.Model() as model:
#prior
    sigma = pm.Exponential('sigma', 50, testval=.1)
    mu = pm.Normal('mu', 0, sigma=3, testval=.1)
    nu = pm.Exponential('nu', 0.15)
    logs = pm.GaussianRandomWalk('logs',tau=sigma**-2, shape=1000)
#likelihood
    r = pm.StudentT('r',nu, mu=mu, lam=1/np.exp(-2*logs), observed=train_set2['logchange'].values)
with model:
    start = pm.find_MAP(vars=[logs, mu, nu,sigma], method="L-BFGS-B")

with model:
    step = pm.NUTS(vars=[logs, mu, nu,sigma] ,scaling=start, gamma=.25)
    start2 = pm.sample(300, step, start=start, progressbar=True)[-1]
    step = pm.NUTS(vars=[logs, mu, nu,sigma],scaling=start2, gamma=.55)
    trace = pm.sample(500,step, start=start2, progressbar=True)
with model:
    predictive_samples = pm.sample_posterior_predictive(trace, samples=48)

In [ ]:
preds_gr = list(predictive_samples.values())[0]
lower = np.percentile(preds_gr, 2.5, axis=1)
upper = np.percentile(preds_gr, 97.5, axis=1)
preds = np.mean(preds_gr, axis=1)
u=mgd["y"].values[10000]
predclose=[]
for i in range(len(preds)):
    u=u*(math.exp(preds[i]))
    predclose.append(u)
mse = mean_squared_error(test_set['y'].values, predclose)
print(mse)
# Set figure size and style
plt.figure(figsize=(12, 6))
plt.style.use('seaborn-whitegrid')

# Plot historical data
plt.plot(range(48), test_set['logchange'].values, marker='o', label='historical', color='#39304A')

# Plot predictions
plt.plot(range(48), preds, marker='o', label='prediction mean', color='#FFAA00')
plt.plot(range(48), lower, marker='.', label='prediction lower bound', color='#FF0000')
plt.plot(range(48), upper, marker='.', label='prediction upper bound', color='#FF0000')
# Labels and title
plt.title('MCMC prediction on log of change', fontsize=16)
plt.xlabel('Timestep', fontsize=12)
plt.ylabel('log change', fontsize=12)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=2)
plt.grid(True, color='lightgrey')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.style.use('seaborn-whitegrid')

# Plot historical data
plt.plot(range(48), test_set['y'].values, marker='o', label='historical', color='#39304A')

# Plot predictions
plt.plot(range(48), predclose, marker='o', label='predictions', color='#FFAA00')

# Labels and title
plt.title('MCMC prediction on electronic demand', fontsize=16)
plt.xlabel('Timestep', fontsize=12)
plt.ylabel('electronic demand', fontsize=12)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=2)
plt.grid(True, color='lightgrey')

plt.tight_layout()
plt.show()

In [ ]:
with pm.Model() as model:
#prior
    sigma = pm.Exponential('sigma', 50, testval=.1)
    mu = pm.Normal('mu', 0, sigma=3, testval=.1)
    nu = pm.Exponential('nu', 0.15)
    logs = pm.GaussianRandomWalk('logs',tau=sigma**-2, shape=1000)
#likelihood
    r = pm.StudentT('r',nu, mu=mu, lam=1/np.exp(-2*logs), observed=train_set2['logchange'].values)
with model:
    start = pm.find_MAP(vars=[logs, mu, nu,sigma], method="L-BFGS-B")

with model:
    step = pm.NUTS(vars=[logs, mu, nu,sigma] ,scaling=start, gamma=.25)
    start2 = pm.sample(300, step, start=start, progressbar=True)[-1]
    step = pm.NUTS(vars=[logs, mu, nu,sigma],scaling=start2, gamma=.55)
    trace = pm.sample(500,step, start=start2, progressbar=True)
with model:
    predictive_samples = pm.sample_posterior_predictive(trace, samples=48)

In [ ]:
preds_gr = list(predictive_samples.values())[0]
lower = np.percentile(preds_gr, 2.5, axis=1)
upper = np.percentile(preds_gr, 97.5, axis=1)
preds = np.mean(preds_gr, axis=1)
u=mgd["y"].values[10000]
predclose=[]
for i in range(len(preds)):
    u=u*(math.exp(preds[i]))
    predclose.append(u)
mse = mean_squared_error(test_set['y'].values, predclose)
print(mse)
# Set figure size and style
plt.figure(figsize=(12, 6))
plt.style.use('seaborn-whitegrid')

# Plot historical data
plt.plot(range(48), test_set['logchange'].values, marker='o', label='historical', color='#39304A')

# Plot predictions
plt.plot(range(48), preds, marker='o', label='prediction mean', color='#FFAA00')
plt.plot(range(48), lower, marker='.', label='prediction lower bound', color='#FF0000')
plt.plot(range(48), upper, marker='.', label='prediction upper bound', color='#FF0000')
# Labels and title
plt.title('MCMC prediction on log of change', fontsize=16)
plt.xlabel('Timestep', fontsize=12)
plt.ylabel('log change', fontsize=12)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=2)
plt.grid(True, color='lightgrey')

plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(12, 6))
plt.style.use('seaborn-whitegrid')

# Plot historical data
plt.plot(range(48), test_set['y'].values, marker='o', label='historical', color='#39304A')

# Plot predictions
plt.plot(range(48), predclose, marker='o', label='predictions', color='#FFAA00')

# Labels and title
plt.title('MCMC prediction on electronic demand', fontsize=16)
plt.xlabel('Timestep', fontsize=12)
plt.ylabel('electronic demand', fontsize=12)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=2)
plt.grid(True, color='lightgrey')

plt.tight_layout()
plt.show()